In [1]:
import os
os.chdir('../../')

In [2]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from scr.util import *

In [47]:
# トレーニングデータセットに対して、オーバーサンプリングを実施

# mean
df_train_mean_gb = pd.read_csv('data/feature_engineered/null_representative/train_null_mean.csv')
df_test_mean_gb = pd.read_csv('data/feature_engineered/null_representative/test_null_mean.csv')
df_train_mean_nn = pd.read_csv('data/feature_engineered/null_representative/train_null_mean_scaled.csv')

# median
df_train_median_gb = pd.read_csv('data/feature_engineered/null_representative/train_null_median.csv')
df_test_median_gb = pd.read_csv('data/feature_engineered/null_representative/test_null_median.csv')
df_train_median_nn = pd.read_csv('data/feature_engineered/null_representative/train_null_median_scaled.csv')

In [120]:
print(len(df_train_mean_gb.columns))
print(len(df_test_mean_gb.columns))

114
113


# オーバーサンプリング

In [121]:
df_train_mean_gb = mapping_columns_if_exist(df_train_mean_gb)
df_test_mean_gb = mapping_columns_if_exist(df_test_mean_gb)
df_train_median_gb = mapping_columns_if_exist(df_train_median_gb)
df_test_median_gb = mapping_columns_if_exist(df_test_median_gb)

def handle_unknown_label(train_series, test_series):
    unique_labels = train_series.unique()
    label_map = {label: idx for idx, label in enumerate(unique_labels)}
    train_encoded = train_series.map(label_map)
    test_encoded = test_series.map(lambda x: label_map.get(x, -1))
    return train_encoded, test_encoded

# mean
combination_mean = df_train_mean_gb.columns[44:110]
for col in combination_mean:
    df_train_mean_gb[col], df_test_mean_gb[col] = handle_unknown_label(df_train_mean_gb[col], df_test_mean_gb[col])

# median
combination_median = df_train_median_gb.columns[44:110]
for col in combination_mean:
    df_train_median_gb[col], df_test_median_gb[col] = handle_unknown_label(df_train_median_gb[col], df_test_median_gb[col])

In [122]:
print(len(df_train_mean_gb.columns))
print(len(df_test_mean_gb.columns))

114
113


In [123]:
df_test_mean_gb.to_csv('data/sampling/over_sampling/smote/test_mean_gb.csv', index=False)
df_test_median_gb.to_csv('data/sampling/over_sampling/smote/test_median_gb.csv', index=False)

In [111]:
# mean
X_mean_gb = df_train_mean_gb.drop(columns=['ProdTaken'], axis=1)
y_mean_gb = df_train_mean_gb['ProdTaken']
X_mean_nn = df_train_mean_nn.drop(columns=['ProdTaken'], axis=1)
y_mean_nn = df_train_mean_nn['ProdTaken']

# median
X_median_gb = df_train_median_gb.drop(columns=['ProdTaken'], axis=1)
y_median_gb = df_train_median_gb['ProdTaken']
X_median_nn = df_train_median_nn.drop(columns=['ProdTaken'], axis=1)
y_median_nn = df_train_median_nn['ProdTaken']

## SMOTE

In [112]:
smote = SMOTE(random_state=0)

# mean
X_mean_gb, y_mean_gb = smote.fit_resample(X_mean_gb, y_mean_gb)
X_mean_nn, y_mean_nn = smote.fit_resample(X_mean_nn, y_mean_nn)

# median
X_median_gb, y_median_gb = smote.fit_resample(X_median_gb, y_median_gb)
X_median_nn, y_median_nn = smote.fit_resample(X_median_nn, y_median_nn)

In [113]:
df_mean_gb_smote = pd.concat([X_mean_gb, y_mean_gb], axis=1)
df_mean_nn_smote = pd.concat([X_mean_nn, y_mean_nn], axis=1)
df_median_gb_smote = pd.concat([X_median_gb, y_median_gb], axis=1)
df_median_nn_smote = pd.concat([X_median_nn, y_median_nn], axis=1)

In [114]:
df_mean_gb_smote.to_csv('data/sampling/over_sampling/smote/train_mean_gb_smote.csv', index=False)
df_mean_nn_smote.to_csv('data/sampling/over_sampling/smote/train_mean_nn_smote.csv', index=False)
df_median_gb_smote.to_csv('data/sampling/over_sampling/smote/train_median_gb_smote.csv', index=False)
df_median_nn_smote.to_csv('data/sampling/over_sampling/smote/train_median_nn_smote.csv', index=False)

In [6]:
df_mean_gb_smote = pd.read_csv('data/sampling/over_sampling/smote/train_mean_gb_smote.csv')
df_mean_gb = pd.read_csv('data/sampling/over_sampling/smote/test_mean_gb.csv')

## ADASYN

# アンダーサンプリング

In [48]:
positive = df_train_mean_gb[df_train_mean_gb['ProdTaken'] == 1]
negative = df_train_mean_gb[df_train_mean_gb['ProdTaken'] == 0]

np.random.seed(0)
shuffle = np.random.permutation(negative.index)
splits = np.array_split(negative.loc[shuffle], 5)

for i in range(5):
    display(splits[i].iloc[:5, 0:20])

df_train_mean_gb_list = []
for i in range(5):
    df_train_mean_gb_split = pd.concat([positive, splits[i]], axis=0)
    df_train_mean_gb_list.append(df_train_mean_gb_split)

for i, df in enumerate(df_train_mean_gb_list):
    df.to_csv(f'data/sampling/under_sampling/mean_gb/train_{i}.csv', index=False)

c:\Users\ricke\anaconda3\envs\competition\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child
1115,1115,51,Self Enquiry,1,6.867974,Small Business,female,1,3,Standard,4,20,0,2,Executive,12.494895,0,Married,Has Car,0_child
1656,1656,26,Self Enquiry,3,6.867974,Large Business,female,3,4,Basic,4,2,0,2,Executive,12.501782,0,Married,Has Car,1_child
1231,1231,58,Company Invited,2,6.734592,Small Business,female,2,4,Deluxe,3,20,0,1,AVP,13.139134,0,Divorced,No Car,1_child
1838,1838,30,Company Invited,1,6.985642,Salaried,female,3,5,Deluxe,3,3,0,2,Manager,12.665840,0,Married,No Car,1_child
1658,1658,44,Company Invited,2,6.042633,Small Business,male,3,4,Standard,3,1,0,1,Senior Manager,12.959129,0,Single,Has Car,2_child


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child
872,872,37,Company Invited,3,6.042633,Salaried,female,2,5,Standard,4,2,0,3,Senior Manager,13.062552,0,Married,Has Car,1_child
342,342,36,Self Enquiry,1,6.398595,Small Business,female,3,4,Basic,3,20,0,1,Executive,12.678639,0,Divorced,No Car,2_child
1289,1289,39,Self Enquiry,2,5.888878,Salaried,female,2,3,Basic,3,2,0,3,Executive,12.473428,0,Divorced,Has Car,1_child
454,454,50,Self Enquiry,2,6.803505,Salaried,male,2,3,King,3,16,0,3,VP,13.123375,0,Single,Has Car,0_child
2882,2882,29,Self Enquiry,3,6.660575,Large Business,female,3,3,Standard,4,8,0,2,Senior Manager,12.899222,0,Single,No Car,2_child


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child
3412,3412,52,Company Invited,2,6.660575,Salaried,female,3,3,Standard,4,2,0,1,Senior Manager,12.905142,0,Divorced,Has Car,1_child
336,336,23,Self Enquiry,1,6.734592,Salaried,male,3,4,Basic,3,7,1,2,Executive,12.632764,0,Single,Has Car,1_child
297,297,54,Self Enquiry,2,6.867974,Small Business,male,3,4,Standard,4,20,0,2,Senior Manager,12.860586,0,Single,Has Car,2_child
2590,2590,27,Company Invited,2,6.293419,Small Business,male,3,4,Deluxe,4,7,0,3,Manager,12.706851,0,Single,No Car,2_child
2642,2642,20,Self Enquiry,1,6.867974,Salaried,male,2,5,Basic,5,2,0,1,Executive,12.464143,0,Married,Has Car,1_child


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child
2902,2902,30,Company Invited,1,7.139660,Salaried,male,2,3,Basic,4,7,0,2,Senior Manager,12.773404,0,Single,No Car,1_child
1500,1500,38,Company Invited,1,6.803505,Salaried,female,4,5,Basic,3,3,0,2,Executive,12.681440,0,Married,No Car,3_child
977,977,48,Self Enquiry,2,5.888878,Small Business,male,2,3,Deluxe,3,1,0,2,Manager,12.611541,0,Married,No Car,1_child
269,269,39,Self Enquiry,3,6.493754,Small Business,male,2,5,Deluxe,3,2,0,2,Manager,12.715946,0,Single,No Car,1_child
281,281,40,Self Enquiry,1,6.803505,Small Business,male,2,4,Basic,3,12,0,2,Executive,12.479894,0,Single,No Car,1_child


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child
557,557,29,Self Enquiry,2,6.867974,Small Business,female,3,3,Basic,3,2,0,4,Executive,12.484730,0,Married,Has Car,0_child
2292,2292,38,Self Enquiry,1,6.580639,Small Business,female,4,4,Basic,4,3,0,3,Executive,12.696801,0,Single,No Car,2_child
806,806,43,Self Enquiry,1,6.175867,Small Business,female,3,4,Standard,3,2,0,4,Senior Manager,12.857032,0,Divorced,No Car,2_child
1443,1443,42,Company Invited,1,6.734592,Large Business,female,2,4,Standard,4,4,0,2,Senior Manager,13.000977,0,Single,No Car,0_child
3411,3411,47,Company Invited,2,6.928538,Small Business,male,3,3,Basic,4,2,0,2,Executive,12.474117,0,Divorced,No Car,2_child


In [49]:
positive = df_train_mean_nn[df_train_mean_gb['ProdTaken'] == 1]
negative = df_train_mean_nn[df_train_mean_gb['ProdTaken'] == 0]

np.random.seed(0)
shuffle = np.random.permutation(negative.index)
splits = np.array_split(negative.loc[shuffle], 5)

for i in range(5):
    display(splits[i].iloc[:5, 0:20])

df_train_mean_gb_list = []
for i in range(5):
    df_train_mean_gb_split = pd.concat([positive, splits[i]], axis=0)
    df_train_mean_gb_list.append(df_train_mean_gb_split)

for i, df in enumerate(df_train_mean_gb_list):
    df.to_csv(f'data/sampling/under_sampling/mean_nn/train_{i}.csv', index=False)

c:\Users\ricke\anaconda3\envs\competition\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,id,Age,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,MonthlyIncome,Motivation,EconomicPower,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,Monetary,ContractRate_FM,ContractRate_G1
1115,1115,-0.106231,0.221793,-1.908669,-0.669802,3.376126,-1.179874,-1.427745,0.204383,-0.027643,0.873620,-0.788227,-1.333079,2.647560,2.886618,11,-0.609549,2.524408,-0.718553,-0.495294
1656,1656,-0.107089,0.221793,0.815826,0.370919,-0.562343,-1.155459,0.414263,-2.048653,-0.248855,1.164827,1.234264,0.283581,-0.729951,0.080058,25,-0.609549,-0.766735,-0.546159,1.202695
1231,1231,0.833799,-0.067486,-0.546422,0.370919,3.376126,2.015706,-0.404408,-0.806247,-0.580672,0.593948,0.637917,0.304298,2.647560,0.942164,10,-0.609549,2.789986,-0.718553,-0.660650
1838,1838,-0.107089,0.511072,0.815826,1.411640,-0.343539,-0.521266,1.028265,0.250632,-0.691277,1.431508,-0.939906,0.368301,-0.230622,-0.652262,16,-0.609549,-0.248960,-0.546159,-0.843906
1658,1658,-0.107088,-1.079962,0.815826,0.370919,-0.781147,0.907307,0.414263,-0.137497,-0.470066,0.661963,-0.251857,-0.240715,-1.433715,-1.318740,3,1.106236,-1.410172,2.836845,-0.397583


,id,Age,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,MonthlyIncome,Motivation,EconomicPower,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,Monetary,ContractRate_FM,ContractRate_G1
872,872,-0.107089,-1.079962,-0.546422,1.411640,-0.562343,1.519666,0.004928,-1.998081,-0.580672,2.482362,1.563241,1.135294,-0.729951,0.080058,19,-0.609549,-0.683318,-0.546159,-0.834008
342,342,-0.107089,-0.646043,0.815826,0.370919,3.376126,-0.467275,0.414263,0.254095,-0.248855,0.625137,-0.934692,-1.261915,2.647560,0.942164,11,1.106236,2.600154,-0.718553,0.543576
1289,1289,-0.107089,-1.224601,-0.546422,-0.669802,-0.562343,-1.254905,-0.813743,-0.926317,0.082962,1.528305,0.018430,-0.267839,-0.729951,-0.928875,12,1.106236,-0.770953,-0.546159,0.543576
454,454,-0.106774,0.077154,-0.546422,-0.669802,2.500911,1.910503,-0.813743,-0.098985,0.414779,1.322848,0.950301,2.298814,2.280791,0.738986,30,-2.325333,2.408007,-0.718553,-0.358557
2882,2882,-0.107089,-0.212125,0.815826,-0.669802,0.750480,0.580517,-0.199740,-2.012811,-0.470066,0.900823,1.782730,1.699949,1.176907,1.664571,27,-0.609549,1.211207,0.078726,-0.113907


,id,Age,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,MonthlyIncome,Motivation,EconomicPower,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,Monetary,ContractRate_FM,ContractRate_G1
3412,3412,-0.104757,-0.212125,0.815826,-0.669802,-0.562343,0.611945,-0.199740,-0.848451,-0.691277,0.450412,0.230191,1.047233,-0.729951,0.080058,20,-0.609549,-0.706734,-0.546159,-0.495294
336,336,-0.107089,-0.067486,0.815826,0.370919,0.531676,-0.657632,2.460938,1.267019,0.414779,1.187897,-0.729617,-0.192414,0.972472,0.014216,13,1.106236,0.930465,0.078726,2.437851
297,297,-0.089856,0.221793,0.815826,0.370919,3.376126,0.379904,0.414263,-0.160603,-0.470066,1.164827,-0.296327,-0.285820,2.647560,2.886618,3,-0.609549,2.675159,-0.718553,-0.264132
2590,2590,-0.107089,-0.790683,0.815826,0.370919,0.531676,-0.345796,0.414263,-0.196650,-0.470066,1.906754,-0.126776,-0.803588,0.972472,1.494694,21,1.106236,0.951324,0.078726,-0.365371
2642,2642,-0.107089,0.221793,-0.546422,1.411640,-0.562343,-1.286863,0.004928,1.207713,0.082962,0.728017,-1.225191,-0.272234,-0.729951,1.088991,13,-2.325333,-0.772335,-0.546159,2.437851


,id,Age,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,MonthlyIncome,Motivation,EconomicPower,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,Monetary,ContractRate_FM,ContractRate_G1
2902,2902,-0.107089,0.944990,-0.546422,-0.669802,0.531676,-0.045273,-0.813743,1.316484,1.078413,0.840376,-0.423007,0.973534,0.972472,1.494694,1,-0.609549,0.970063,0.078726,1.228560
1500,1500,-0.107089,0.077154,2.178073,1.411640,-0.343539,-0.455365,2.051603,0.254853,-0.801883,1.469831,-1.671606,-0.169372,-0.230622,-0.652262,13,1.106236,-0.246032,-0.546159,0.543576
977,977,-0.107046,-1.224601,-0.546422,-0.669802,-0.781147,-0.742787,-0.813743,-0.218998,-0.580672,1.018870,-0.171579,-0.843857,-1.433715,-1.318740,21,-0.609549,-1.442794,2.836845,-0.821803
269,269,-0.107089,-0.501404,-0.546422,1.411640,-0.562343,-0.305898,0.004928,-1.685310,-0.248855,1.539941,2.377155,-0.799745,-0.729951,-0.928875,35,-0.609549,-0.734878,-0.546159,-0.678736
281,281,-0.107089,0.077154,-0.546422,0.370919,1.625695,-1.232475,-0.404408,1.213253,1.078413,1.175864,-0.553453,-1.338889,1.815166,0.481043,11,1.106236,1.707837,-0.718553,1.158270


,id,Age,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,MonthlyIncome,Motivation,EconomicPower,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,Monetary,ContractRate_FM,ContractRate_G1
557,557,-0.107089,0.221793,0.815826,-0.669802,-0.562343,-1.215603,-0.199740,-0.924279,0.414779,1.747240,0.023973,-1.337016,-0.729951,-0.928875,5,1.106236,-0.769272,-0.546159,1.202695
2292,2292,-0.107089,-0.356765,2.178073,0.370919,-0.343539,-0.389465,1.232933,0.259008,-0.248855,1.823531,-0.927293,-1.254880,-0.230622,0.494978,11,-0.609549,-0.243149,-0.546159,0.543576
806,806,-0.107089,-0.935322,0.815826,0.370919,-0.562343,0.361841,0.414263,0.302359,-0.691277,2.590729,-0.862015,-0.287446,-0.729951,-0.928875,8,1.106236,-0.713890,-0.546159,-0.687494
1443,1443,-0.107089,-0.067486,-0.546422,0.370919,-0.124735,1.147486,-0.404408,0.341303,0.414779,1.187897,0.255942,1.762050,0.156688,0.816815,7,-0.609549,0.206761,0.368799,-0.687494
3411,3411,-0.107073,0.366432,0.815826,-0.669802,-0.562343,-1.252523,-0.199740,-0.251220,-0.248855,0.865984,-0.236179,-1.341126,-0.729951,0.080058,5,-0.609549,-0.770851,-0.546159,1.158270


In [50]:
positive = df_train_median_gb[df_train_mean_gb['ProdTaken'] == 1]
negative = df_train_median_gb[df_train_mean_gb['ProdTaken'] == 0]

np.random.seed(0)
shuffle = np.random.permutation(negative.index)
splits = np.array_split(negative.loc[shuffle], 5)

for i in range(5):
    display(splits[i].iloc[:5, 0:20])

df_train_mean_gb_list = []
for i in range(5):
    df_train_mean_gb_split = pd.concat([positive, splits[i]], axis=0)
    df_train_mean_gb_list.append(df_train_mean_gb_split)

for i, df in enumerate(df_train_mean_gb_list):
    df.to_csv(f'data/sampling/under_sampling/median_gb/train_{i}.csv', index=False)

c:\Users\ricke\anaconda3\envs\competition\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child
1115,1115,51,Self Enquiry,1,6.867974,Small Business,female,1,3,Standard,4,20,0,2,Executive,12.494895,0,Married,Has Car,0_child
1656,1656,26,Self Enquiry,3,6.867974,Large Business,female,3,4,Basic,4,2,0,2,Executive,12.501782,0,Married,Has Car,1_child
1231,1231,58,Company Invited,2,6.734592,Small Business,female,2,4,Deluxe,3,20,0,1,AVP,13.139134,0,Divorced,No Car,1_child
1838,1838,30,Company Invited,1,6.985642,Salaried,female,3,5,Deluxe,3,3,0,2,Manager,12.665840,0,Married,No Car,1_child
1658,1658,44,Company Invited,2,6.042633,Small Business,male,3,4,Standard,3,1,0,1,Senior Manager,12.959129,0,Single,Has Car,2_child


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child
872,872,37,Company Invited,3,6.042633,Salaried,female,2,5,Standard,4,2,0,3,Senior Manager,13.062552,0,Married,Has Car,1_child
342,342,36,Self Enquiry,1,6.398595,Small Business,female,3,4,Basic,3,20,0,1,Executive,12.678639,0,Divorced,No Car,2_child
1289,1289,39,Self Enquiry,2,5.888878,Salaried,female,2,3,Basic,3,2,0,3,Executive,12.473428,0,Divorced,Has Car,1_child
454,454,50,Self Enquiry,2,6.803505,Salaried,male,2,3,King,3,16,0,3,VP,13.123375,0,Single,Has Car,0_child
2882,2882,29,Self Enquiry,3,6.660575,Large Business,female,3,3,Standard,4,8,0,2,Senior Manager,12.899222,0,Single,No Car,2_child


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child
3412,3412,52,Company Invited,2,6.660575,Salaried,female,3,3,Standard,4,2,0,1,Senior Manager,12.905142,0,Divorced,Has Car,1_child
336,336,23,Self Enquiry,1,6.734592,Salaried,male,3,4,Basic,3,7,1,2,Executive,12.632764,0,Single,Has Car,1_child
297,297,54,Self Enquiry,2,6.867974,Small Business,male,3,4,Standard,4,20,0,2,Senior Manager,12.860586,0,Single,Has Car,2_child
2590,2590,27,Company Invited,2,6.293419,Small Business,male,3,4,Deluxe,4,7,0,3,Manager,12.706851,0,Single,No Car,2_child
2642,2642,20,Self Enquiry,1,6.867974,Salaried,male,2,5,Basic,5,2,0,1,Executive,12.464143,0,Married,Has Car,1_child


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child
2902,2902,30,Company Invited,1,7.139660,Salaried,male,2,3,Basic,4,7,0,2,Senior Manager,12.773404,0,Single,No Car,1_child
1500,1500,38,Company Invited,1,6.803505,Salaried,female,4,5,Basic,3,3,0,2,Executive,12.681440,0,Married,No Car,3_child
977,977,48,Self Enquiry,2,5.888878,Small Business,male,2,3,Deluxe,3,1,0,2,Manager,12.611541,0,Married,No Car,1_child
269,269,39,Self Enquiry,3,6.493754,Small Business,male,2,5,Deluxe,3,2,0,2,Manager,12.715946,0,Single,No Car,1_child
281,281,40,Self Enquiry,1,6.803505,Small Business,male,2,4,Basic,3,12,0,2,Executive,12.479894,0,Single,No Car,1_child


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child
557,557,29,Self Enquiry,2,6.867974,Small Business,female,3,3,Basic,3,2,0,4,Executive,12.484730,0,Married,Has Car,0_child
2292,2292,38,Self Enquiry,1,6.580639,Small Business,female,4,4,Basic,4,3,0,3,Executive,12.696801,0,Single,No Car,2_child
806,806,43,Self Enquiry,1,6.175867,Small Business,female,3,4,Standard,3,2,0,4,Senior Manager,12.857032,0,Divorced,No Car,2_child
1443,1443,42,Company Invited,1,6.734592,Large Business,female,2,4,Standard,4,4,0,2,Senior Manager,13.000977,0,Single,No Car,0_child
3411,3411,47,Company Invited,2,6.928538,Small Business,male,3,3,Basic,4,2,0,2,Executive,12.474117,0,Divorced,No Car,2_child


In [51]:
positive = df_train_mean_nn[df_train_median_nn['ProdTaken'] == 1]
negative = df_train_mean_nn[df_train_median_nn['ProdTaken'] == 0]

np.random.seed(0)
shuffle = np.random.permutation(negative.index)
splits = np.array_split(negative.loc[shuffle], 5)

for i in range(5):
    display(splits[i].iloc[:5, 0:20])

df_train_mean_gb_list = []
for i in range(5):
    df_train_mean_gb_split = pd.concat([positive, splits[i]], axis=0)
    df_train_mean_gb_list.append(df_train_mean_gb_split)

for i, df in enumerate(df_train_mean_gb_list):
    df.to_csv(f'data/sampling/under_sampling/median_nn/train_{i}.csv', index=False)

c:\Users\ricke\anaconda3\envs\competition\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,id,Age,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,MonthlyIncome,Motivation,EconomicPower,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,Monetary,ContractRate_FM,ContractRate_G1
1115,1115,-0.106231,0.221793,-1.908669,-0.669802,3.376126,-1.179874,-1.427745,0.204383,-0.027643,0.873620,-0.788227,-1.333079,2.647560,2.886618,11,-0.609549,2.524408,-0.718553,-0.495294
1656,1656,-0.107089,0.221793,0.815826,0.370919,-0.562343,-1.155459,0.414263,-2.048653,-0.248855,1.164827,1.234264,0.283581,-0.729951,0.080058,25,-0.609549,-0.766735,-0.546159,1.202695
1231,1231,0.833799,-0.067486,-0.546422,0.370919,3.376126,2.015706,-0.404408,-0.806247,-0.580672,0.593948,0.637917,0.304298,2.647560,0.942164,10,-0.609549,2.789986,-0.718553,-0.660650
1838,1838,-0.107089,0.511072,0.815826,1.411640,-0.343539,-0.521266,1.028265,0.250632,-0.691277,1.431508,-0.939906,0.368301,-0.230622,-0.652262,16,-0.609549,-0.248960,-0.546159,-0.843906
1658,1658,-0.107088,-1.079962,0.815826,0.370919,-0.781147,0.907307,0.414263,-0.137497,-0.470066,0.661963,-0.251857,-0.240715,-1.433715,-1.318740,3,1.106236,-1.410172,2.836845,-0.397583


,id,Age,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,MonthlyIncome,Motivation,EconomicPower,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,Monetary,ContractRate_FM,ContractRate_G1
872,872,-0.107089,-1.079962,-0.546422,1.411640,-0.562343,1.519666,0.004928,-1.998081,-0.580672,2.482362,1.563241,1.135294,-0.729951,0.080058,19,-0.609549,-0.683318,-0.546159,-0.834008
342,342,-0.107089,-0.646043,0.815826,0.370919,3.376126,-0.467275,0.414263,0.254095,-0.248855,0.625137,-0.934692,-1.261915,2.647560,0.942164,11,1.106236,2.600154,-0.718553,0.543576
1289,1289,-0.107089,-1.224601,-0.546422,-0.669802,-0.562343,-1.254905,-0.813743,-0.926317,0.082962,1.528305,0.018430,-0.267839,-0.729951,-0.928875,12,1.106236,-0.770953,-0.546159,0.543576
454,454,-0.106774,0.077154,-0.546422,-0.669802,2.500911,1.910503,-0.813743,-0.098985,0.414779,1.322848,0.950301,2.298814,2.280791,0.738986,30,-2.325333,2.408007,-0.718553,-0.358557
2882,2882,-0.107089,-0.212125,0.815826,-0.669802,0.750480,0.580517,-0.199740,-2.012811,-0.470066,0.900823,1.782730,1.699949,1.176907,1.664571,27,-0.609549,1.211207,0.078726,-0.113907


,id,Age,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,MonthlyIncome,Motivation,EconomicPower,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,Monetary,ContractRate_FM,ContractRate_G1
3412,3412,-0.104757,-0.212125,0.815826,-0.669802,-0.562343,0.611945,-0.199740,-0.848451,-0.691277,0.450412,0.230191,1.047233,-0.729951,0.080058,20,-0.609549,-0.706734,-0.546159,-0.495294
336,336,-0.107089,-0.067486,0.815826,0.370919,0.531676,-0.657632,2.460938,1.267019,0.414779,1.187897,-0.729617,-0.192414,0.972472,0.014216,13,1.106236,0.930465,0.078726,2.437851
297,297,-0.089856,0.221793,0.815826,0.370919,3.376126,0.379904,0.414263,-0.160603,-0.470066,1.164827,-0.296327,-0.285820,2.647560,2.886618,3,-0.609549,2.675159,-0.718553,-0.264132
2590,2590,-0.107089,-0.790683,0.815826,0.370919,0.531676,-0.345796,0.414263,-0.196650,-0.470066,1.906754,-0.126776,-0.803588,0.972472,1.494694,21,1.106236,0.951324,0.078726,-0.365371
2642,2642,-0.107089,0.221793,-0.546422,1.411640,-0.562343,-1.286863,0.004928,1.207713,0.082962,0.728017,-1.225191,-0.272234,-0.729951,1.088991,13,-2.325333,-0.772335,-0.546159,2.437851


,id,Age,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,MonthlyIncome,Motivation,EconomicPower,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,Monetary,ContractRate_FM,ContractRate_G1
2902,2902,-0.107089,0.944990,-0.546422,-0.669802,0.531676,-0.045273,-0.813743,1.316484,1.078413,0.840376,-0.423007,0.973534,0.972472,1.494694,1,-0.609549,0.970063,0.078726,1.228560
1500,1500,-0.107089,0.077154,2.178073,1.411640,-0.343539,-0.455365,2.051603,0.254853,-0.801883,1.469831,-1.671606,-0.169372,-0.230622,-0.652262,13,1.106236,-0.246032,-0.546159,0.543576
977,977,-0.107046,-1.224601,-0.546422,-0.669802,-0.781147,-0.742787,-0.813743,-0.218998,-0.580672,1.018870,-0.171579,-0.843857,-1.433715,-1.318740,21,-0.609549,-1.442794,2.836845,-0.821803
269,269,-0.107089,-0.501404,-0.546422,1.411640,-0.562343,-0.305898,0.004928,-1.685310,-0.248855,1.539941,2.377155,-0.799745,-0.729951,-0.928875,35,-0.609549,-0.734878,-0.546159,-0.678736
281,281,-0.107089,0.077154,-0.546422,0.370919,1.625695,-1.232475,-0.404408,1.213253,1.078413,1.175864,-0.553453,-1.338889,1.815166,0.481043,11,1.106236,1.707837,-0.718553,1.158270


,id,Age,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,MonthlyIncome,Motivation,EconomicPower,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,Monetary,ContractRate_FM,ContractRate_G1
557,557,-0.107089,0.221793,0.815826,-0.669802,-0.562343,-1.215603,-0.199740,-0.924279,0.414779,1.747240,0.023973,-1.337016,-0.729951,-0.928875,5,1.106236,-0.769272,-0.546159,1.202695
2292,2292,-0.107089,-0.356765,2.178073,0.370919,-0.343539,-0.389465,1.232933,0.259008,-0.248855,1.823531,-0.927293,-1.254880,-0.230622,0.494978,11,-0.609549,-0.243149,-0.546159,0.543576
806,806,-0.107089,-0.935322,0.815826,0.370919,-0.562343,0.361841,0.414263,0.302359,-0.691277,2.590729,-0.862015,-0.287446,-0.729951,-0.928875,8,1.106236,-0.713890,-0.546159,-0.687494
1443,1443,-0.107089,-0.067486,-0.546422,0.370919,-0.124735,1.147486,-0.404408,0.341303,0.414779,1.187897,0.255942,1.762050,0.156688,0.816815,7,-0.609549,0.206761,0.368799,-0.687494
3411,3411,-0.107073,0.366432,0.815826,-0.669802,-0.562343,-1.252523,-0.199740,-0.251220,-0.248855,0.865984,-0.236179,-1.341126,-0.729951,0.080058,5,-0.609549,-0.770851,-0.546159,1.158270
